In [1]:
! pip install mysql-connector-python
import mysql.connector
import pandas as pd

In [2]:
con = open('ref_con/db-prima', 'r').read().splitlines()
how = input('how to connect(local/public):')

mydb = mysql.connector.connect(
  host=con[0] if how == 'public' else con[1],
  user=con[2],
  password=con[3],
  database=con[4] 
)

mycursor = mydb.cursor(dictionary=True)

In [39]:
def exec(q):
  mycursor.execute(q)
  return mycursor.fetchall()

def exc_q_elektif(m):
    mNext = str(int(m) + 1).zfill(2)
    Y = 2022
    YNext = Y if int(m) < 12 else (Y+1)
    m = m.zfill(2)
    print(m,mNext)
    return exec('select no_perjanjian_operasi,tgl_pelaksanaan,catatan_pasca_operasi from pendaftaran_perjanjian_operasi where tgl_pelaksanaan between "2022-'+m+'-01 00:00:00" and "'+str(YNext)+'-'+mNext+'-01 00:00:00" and status = 5 and catatan_pra_operasi not like "%CITO%"')

def do_in_out(Y,dp):
    ret = []
    ret.append(exc_q_transfer(Y,dp,'M'))
    ret.append(exc_q_transfer(Y,dp,'K'))
    return ret

def komparasiDenganDesc(D1,D2,r=0):
    return pd.merge(D1,D2,on=r,how='outer',indicator=True)

#bangun query
def bangun_q(DF,po,selisihM):
    trn = ''
    ret = ''
    for i,r in DF[['kode_obat','qty_diberi','no_transfer','keterangan']]\
[(DF['keterangan'].str.contains(pat=po,regex=True,case=False))].iterrows():
        ret += "update inventori_penerimaan_logistik_farmasi_detail set qty = qty - "+str(r['qty_diberi'])+" \
    where no_po = '"+po+"' and kode_obat = '"+r['kode_obat']+"';\n \
update log_stock_obatdepo set \
    jumlah_mutasi = jumlah_mutasi - "+str(r['qty_diberi'])+" where id_referensi = '"+po+"' \
    and kode_obat = '"+r['kode_obat']+"';\n"
        if(trn != r['no_transfer']):
            trn = r['no_transfer']
            ret += "UPDATE inventori_transfer_logistik\
 SET tgl_buat_permintaan = DATE_ADD(tgl_buat_permintaan, INTERVAL "+selisihM+" MONTH),\
tgl_acc_permintaan = DATE_ADD(tgl_acc_permintaan, INTERVAL "+selisihM+" MONTH) \
WHERE no_transfer = '"+trn+"';\n \
update log_stock_obatdepo set last_update = DATE_ADD(last_update, INTERVAL "+selisihM+" MONTH) \
where id_referensi = '"+trn+"';\n"
    return ret

In [61]:
import json
from datetime import datetime

DF = pd.DataFrame.from_dict(exc_q_elektif('2'))

DF['waktu_masuk_ok'] = DF.iloc[:,1:].astype(str).apply(lambda x: json.loads(x['catatan_pasca_operasi'])['waktu_masuk_ok'] ,axis=1)

DF['selisih(menit)'] = DF.iloc[:,1:].astype(str).apply(lambda x: 'waktu masuk ok kosong' if x['waktu_masuk_ok'] == '' else divmod((datetime.strptime(x['waktu_masuk_ok'],"%Y-%m-%d %H:%M:%S") - datetime.strptime(x['tgl_pelaksanaan'],"%Y-%m-%d %H:%M:%S")).seconds,60)[0] ,axis=1)

DF

02 03


,no_perjanjian_operasi,tgl_pelaksanaan,catatan_pasca_operasi,waktu_masuk_ok,selisih(menit)
0,JO211129017,2022-02-02 08:00:00,"{""waktu_masuk_ok"":""2022-02-02 08:00:00""}",2022-02-02 08:00:00,0
1,JO211223009,2022-02-02 08:00:00,"{""waktu_masuk_ok"":""2022-02-02 08:00:00""}",2022-02-02 08:00:00,0
2,JO211231052,2022-02-02 08:00:00,"{""waktu_masuk_ok"":""2022-02-02 08:00:00""}",2022-02-02 08:00:00,0
3,JO220118029,2022-02-02 08:00:00,"{""waktu_masuk_ok"":""2022-02-02 08:00:00""}",2022-02-02 08:00:00,0
4,JO220121026,2022-02-02 08:00:00,"{""waktu_masuk_ok"":""2022-02-02 08:00:00""}",2022-02-02 08:00:00,0
...,...,...,...,...,...
325,JO220224004,2022-02-25 10:00:00,"{""waktu_masuk_ok"":""2022-02-25 10:00:00""}",2022-02-25 10:00:00,0
326,JO220224010,2022-02-25 10:00:00,"{""waktu_masuk_ok"":""2022-02-25 10:00:00""}",2022-02-25 10:00:00,0
327,JO220216028,2022-02-25 11:00:00,"{""waktu_masuk_ok"":""2022-02-25 11:00:00""}",2022-02-25 11:00:00,0
328,JO220218031,2022-02-25 11:00:00,"{""waktu_masuk_ok"":""2022-02-25 11:00:00""}",2022-02-25 11:00:00,0
